In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("pokemon_data.csv", index_col = "Name")
df.head()

,Passes,Smashes
Name,,
Bulbasaur,1187934,331851
Ivysaur,1172763,303853
Venusaur,1175757,273495
Charmander,1152826,271930
Charmeleon,1135715,275524


In [6]:
df['Smash Normalized'] = (df['Smashes'] - df['Smashes'].min()) / (df['Smashes'].max() - df['Smashes'].min())
df['Pass Normalized'] = (df['Passes'] - df['Passes'].min()) / (df['Passes'].max() - df['Passes'].min())
df

,Passes,Smashes,Smash Normalized,Pass Normalized
Name,,,,
Bulbasaur,1187934,331851,0.408613,1.000000
Ivysaur,1172763,303853,0.373735,0.987044
Venusaur,1175757,273495,0.335917,0.989601
Charmander,1152826,271930,0.333967,0.970017
Charmeleon,1135715,275524,0.338445,0.955404
...,...,...,...,...
Raging-bolt,56363,11343,0.009345,0.033627
Iron-boulder,59467,8227,0.005464,0.036278
Iron-crown,53995,13697,0.012278,0.031604
